In [2]:
# %load bi_lstm_cv.py

%autosave 600

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from keras import layers as lrs
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, Callback, ModelCheckpoint


# get data
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")


# set vars
max_features = 100000
maxlength = 150 
embed_size = 300


# preprocess
print('preprocessing')

class_names = list(train)[-6:]
y_train = train[class_names].values

multarray = np.array([100000, 10000, 1000, 100, 10, 1])
y_multi = np.sum(train[class_names].values * multarray, axis=1)

train['comment_text'].fillna("no comment")
test['comment_text'].fillna("no comment")

x_train0 = train['comment_text']
x_test0 = test['comment_text']

toker = text.Tokenizer(num_words=max_features, 
    filters='?!"#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n',
    lower=True)
toker.fit_on_texts(list(x_train0)+list(x_test0))
x_train = toker.texts_to_sequences(x_train0)
x_test = toker.texts_to_sequences(x_test0)
x_train = sequence.pad_sequences(x_train, maxlen=maxlength)
x_test = sequence.pad_sequences(x_test, maxlen=maxlength)


# prepare word vectors/matrix
print('vectorizing')

embedding_file = "../vecs/glove.840B.300d.txt" 
embeddings_index = {}
with open(embedding_file, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs          
               
word_idx = toker.word_index

num_words = min(max_features, len(word_idx) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_idx.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector  # words not found will be all-zeros




# model
print('modeling')

def create_model():
    model = Sequential()
    model.add(lrs.Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False))
    model.add(lrs.SpatialDropout1D(0.2)) # 0.2
    model.add(lrs.Bidirectional(lrs.LSTM(128, return_sequences=True, dropout=0.0, recurrent_dropout=0.0), # 128 0,0 
        merge_mode='concat'))          # 'concat' 
    model.add(lrs.Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "glorot_uniform"))
    model.add(lrs.GlobalMaxPooling1D())    # avg pooling
    model.add(lrs.Dense(6, activation="sigmoid"))
    model.compile(loss='binary_crossentropy',optimizer=Nadam(lr=0.001),metrics=['accuracy']) # default 0.002
    return model


class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.x_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC_AUC - epoch {:d} - score {:.6f}".format(epoch+1, score))

def get_calls(x_val, y_val, weights):
    ra_val = RocAucEvaluation(validation_data=(x_val, y_val), interval=1)
    checkpoint = ModelCheckpoint(weights, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
    callbacks_list = [ra_val, checkpoint, early]
    return callbacks_list

def fit_model(x_tra, y_tra, x_val, y_val, weights):
    clbks = get_calls(x_val, y_val, weights)
    clf = create_model()
    clf.fit(x_tra, y_tra, batch_size=128, epochs=1, validation_data=(x_val, y_val),     ### 4-5
            callbacks=clbks, verbose=1)
    clf.load_weights(weights)
    return clf
    
# do it
skf = StratifiedKFold(n_splits=2, shuffle=True)                                                  ## 5
for i, (train_index, val_index) in enumerate(skf.split(np.zeros(train.shape[0]), y_multi)):
    print ("\n\n\n Training on fold {} \n\n\n".format(str(i+1)))
    xt, xv = x_train[train_index], x_train[val_index]
    yt, yv = y_train[train_index], y_train[val_index]
    weights_file = "./weights/weights_best_{}.hdf5".format(str(i+1))
    clfr = fit_model(xt, yt, xv, yv, weights_file)


    
    
#cflr.get_weights


# # # average all best weights
# fold_weight_list = glob('./weights/*.hdf5')

# mean_weights = list()

# for weights_list_tuple in zip(*weights):
#     mean_weights.append(
#         [numpy.array(weights_).mean(axis=0)\
#             for weights_ in zip(*weights_list_tuple)])

# new_model.set_weights(mean_weights)





# test_preds = clfr.predict(x_test, batch_size=1024, verbose=1)

# submission = pd.read_csv("../input/sample_submission.csv")
# submission[class_names] = (test_preds)
# submission.to_csv("../subs/sub_bilstm.csv", index = False)




Autosaving every 600 seconds


Using TensorFlow backend.


preprocessing
vectorizing
modeling



 Training on fold 1 





/home/mjp914/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         30000000  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, None, 300)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 256)         439296    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          32832     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
Total params: 30,472,518
Trainable params: 472,518
Non-trainable params: 30,000,000
__________________________________________________________

In [4]:
j = clfr.get_weights

In [5]:
j

<bound method Sequential.get_weights of <keras.models.Sequential object at 0x7f9704554a20>>